In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import chardet
import json

In [ ]:
import os
os.chdir('/content/drive/MyDrive/lab5')

In [ ]:
train_Data = pd.read_csv("train.txt")
test_Index = pd.read_csv("test_without_label.txt")['guid'].values
train_Count = train_Data["tag"].value_counts() * 0.75 // 1   #训练集：验证集 = 3：1
train_Data

,guid,tag
0,4597,negative
1,26,neutral
2,4383,negative
3,212,positive
4,2626,positive
...,...,...
3995,3944,negative
3996,945,positive
3997,695,positive
3998,4874,negative


In [ ]:
train_Neg = train_Data.loc[train_Data["tag"] == "negative"].sample(n=int(train_Count["negative"])).values
train_Pos = train_Data.loc[train_Data["tag"] == "positive"].sample(n=int(train_Count["positive"])).values
train_Neu = train_Data.loc[train_Data["tag"] == "neutral"].sample(n=int(train_Count["neutral"])).values
val_Neg = train_Data[~train_Data["guid"].isin(train_Neg[:, 0])].loc[train_Data["tag"] == "negative"].values
val_Pos = train_Data[~train_Data["guid"].isin(train_Pos[:, 0])].loc[train_Data["tag"] == "positive"].values
val_Neu = train_Data[~train_Data["guid"].isin(train_Neu[:, 0])].loc[train_Data["tag"] == "neutral"].values

In [ ]:
def process_dataset(data,dataset):
    for i in range(len(data)):
        guid = data[i][0]
        label = data[i][1]
        encoding = chardet.detect(open('data/'+str(guid)+'.txt','rb').read())['encoding']
        if encoding in ['GB2312','EUC-JP','Big5','Windows-1253','IBM866']:
            encoding = 'GBK' 
        dataset.append({
            'text':open('data/'+str(guid)+'.txt',encoding = encoding).read().rstrip('\n'),
            'label':label,
            'img':'data/'+str(guid)+'.jpg'
        })
    return dataset

In [ ]:
train_dataset = []
train_dataset = process_dataset(train_Neg, train_dataset)
train_dataset = process_dataset(train_Pos, train_dataset)
train_dataset = process_dataset(train_Neu, train_dataset)

In [ ]:
val_dataset = []
val_dataset = process_dataset(val_Neg, val_dataset)
val_dataset = process_dataset(val_Pos, val_dataset)
val_dataset = process_dataset(val_Neu, val_dataset)

In [ ]:
test_dataset=[]
for guid in test_Index:
    encoding = chardet.detect(open('data/'+str(guid)+'.txt','rb').read())['encoding']
    if encoding in ['GB2312','EUC-JP','Big5','Windows-1253','Windows-1254']:
        encoding = 'GBK' 
    test_dataset.append({
        'text':open('data/'+str(guid)+'.txt',encoding = encoding).read().rstrip('\n'),
        'label':None,
        'img':'data/'+str(guid)+'.jpg'        
    })

In [ ]:
with open("train.json", "w", encoding="utf-8") as f:
    json.dump(train_dataset, f)
with open("val.json", "w", encoding="utf-8") as f:
    json.dump(val_dataset, f)
with open("test.json", "w", encoding="utf-8") as f:
    json.dump(test_dataset, f)

In [ ]:
print('Number of train data:', len(train_dataset))
print('Number of validation data:', len(val_dataset))
print('Number of test data:', len(test_dataset))

Number of train data: 2999
Number of validation data: 1001
Number of test data: 511
